# [Day 16](https://adventofcode.com/2022/day/16)

## Read input file

In [1]:
import re

class Valve:
    def __init__(self, name, pressure, connections):
        self.name = name
        self.pressure = pressure
        self.connections = connections
        self.closed = True

valves = dict()
values = []
with open('input.txt', 'r') as file:
    values = [re.findall('Valve ([^ ]+) has flow rate=(\d+); tunnels? leads? to valves? (.+)', line) for line in file.read().strip().split('\n')]
for v in values:
    valves[v[0][0]] = Valve(v[0][0], int(v[0][1]), v[0][2].split(', '))

## Part 1

In [2]:
distances = dict()

def get_min_distance(v1, v2, valves):
    if (v1,v2) in distances:
        return distances[(v1,v2)]
    if (v2,v1) in distances:
        return distances[(v2,v1)]

    q = [(v1, 0)]
    while len(q) > 0:
        v, d = q.pop(0)
        if v == v2:
            distances[(v1,v2)] = d
            return d
        q.extend([(valves[x], d+1) for x in v.connections if x in valves])

# I'm sick today, so full search it is
def search(valve, valves, time_left):
    return search_internal(valve, valves, time_left, 0)
    
def search_internal(valve, valves, time_left, pressure):
    if time_left <= 0:
        return pressure
    
    closed = [valves[x] for x in valves if valves[x].closed and valves[x].pressure > 0 and time_left - get_min_distance(valve, valves[x], valves) >= 2]
    if len(closed) == 0:
        return pressure
    best = 0
    for v in closed:
        v.closed = False
        t = time_left - get_min_distance(valve, v, valves)
        p = pressure + v.pressure*(t-1)
        best = max(best, search_internal(v, valves, t-1, p))
        v.closed = True
    return best
    
    

In [3]:
for v in valves:
    valves[v].closed = True

time_left = 30
pressure = 0
start = valves['AA']
search(start, valves, 30)

1915

## Part 2

In [4]:
# full search again, with a simple memory to at least return after several minutes
def search_with_elephant(valves, start_valve, start_valve_elephant, time_left):
    return search_with_elephant_internal(valves, dict(), start_valve, start_valve_elephant, time_left, time_left)

def search_with_elephant_internal(valves, memory, valve, valve_elephant, time_left, time_left_elephant):
    closed = [valves[x] for x in valves if valves[x].closed and valves[x].pressure > 0 and time_left - get_min_distance(valve, valves[x], valves) >= 2]
    closed_elephant = [valves[x] for x in valves if valves[x].closed and valves[x].pressure > 0 and time_left_elephant - get_min_distance(valve_elephant, valves[x], valves) >= 2]

    if len(closed) == 0 and len(closed_elephant) == 0:
        return []
    best_sum = 0
    best = []
    
    
    for v in closed:
        v.closed = False
        t = time_left - get_min_distance(valve, v, valves)
        if t > 1:
            key = (''.join(sorted([x.name for x in valves.values() if x.closed and x.pressure > 0])), (v.name, valve_elephant.name), (t-1, time_left_elephant))
            reverse_key = (key[0], (key[1][1], key[1][0]), key[2][1], key[2][0])
            sequence = []
            if key in memory:
                sequence = memory[key].copy()
            elif reverse_key in memory:
                sequence = memory[reverse_key].copy()
            else:
                sequence = search_with_elephant_internal(valves, memory, v, valve_elephant, t-1, time_left_elephant)
                memory[key] = sequence.copy()
            sequence.append((t-1, v.pressure, v.name))
            s = sum([x[0]*x[1] for x in sequence])
            if s > best_sum:
                best_sum = s
                best = sequence
        v.closed = True

    for v in closed_elephant:
        v.closed = False
        t = time_left_elephant - get_min_distance(valve_elephant, v, valves)
        if t > 1:
            key = (''.join(sorted([x.name for x in valves.values() if x.closed and x.pressure > 0])), (valve.name, v.name), (time_left, t-1))
            reverse_key = (key[0], (key[1][1], key[1][0]), key[2][1], key[2][0])
            sequence = []
            if key in memory:
                sequence = memory[key].copy()
            elif reverse_key in memory:
                sequence = memory[reverse_key].copy()
            if key in memory:
                sequence = memory[key].copy()
            else:
                sequence = search_with_elephant_internal(valves, memory, valve, v, time_left, t-1)
                memory[key] = sequence.copy()
            sequence.append((t-1, v.pressure, v.name))
            s = sum([x[0]*x[1] for x in sequence])
            if s > best_sum:
                best_sum = s
                best = sequence
        v.closed = True
    return best
    
    
for v in valves:
    valves[v].closed = True
distances = dict()
time_left = 26
pressure = 0
start = valves['AA']
sequence = search_with_elephant(valves, start, start, 26)

display(sorted([(26-x[0], x[1], x[2]) for x in sequence], key=lambda x: x[0]), sum([x[0]*x[1] for x in sequence]))

[(3, 8, 'FW'),
 (4, 10, 'XD'),
 (6, 12, 'KE'),
 (7, 20, 'LU'),
 (9, 17, 'FE'),
 (10, 25, 'KS'),
 (12, 19, 'LW'),
 (13, 22, 'YC'),
 (15, 24, 'VS'),
 (19, 16, 'SX'),
 (21, 15, 'WX'),
 (22, 14, 'HQ')]

2772